# First steps with word embeddings
Big Data and Automated Content Analysis

Damian Trilling


This notebook shows you some first steps of how to work with word embeddings in Gensim. 


**NB  Some of these things may take a lot of memory and/or computing power. **

## Training word embeddings
Training word embedings with gensim is very simple (see example code below - even though you wildo l need to specift some extra options). However, you need a massive dataset for that (think of millions of documents). We're therefore not going to do this in class.

```
model = gensim.models.Word2Vec()
model.build_vocab(sentences)
model.train(sentences)
```

## Using pre-trained models
We can use pre-trained embeddings instead. You can either download them yourself and then read them from a file (which you probably want to do if you want, for instance, use our Dutch embeddings that we talked about). 
But we can also use some embeddings that come with gensim and that gensim downloads for us.

In [28]:
import gensim.downloader as api

model = api.load("glove-wiki-gigaword-100")

# alternative that is 16 times as big:
# model = api.load("word2vec-google-news-300")

In [7]:
# model = api.load("word2vec-google-news-300")

[==================================================] 100.0% 1662.8/1662.8MB downloaded


## Word similarities
Let's play around with word similarieties.

In [13]:
model.most_similar('cat')

[('dog', 0.8798074722290039),
 ('rabbit', 0.7424426674842834),
 ('cats', 0.7323004007339478),
 ('monkey', 0.7288709878921509),
 ('pet', 0.7190139889717102),
 ('dogs', 0.7163872718811035),
 ('mouse', 0.6915250420570374),
 ('puppy', 0.6800068020820618),
 ('rat', 0.6641027331352234),
 ('spider', 0.6501135230064392)]

In [35]:
animals = ['cat', 'dog', 'horse', 'goldfish', 'lion']
for animal in animals:
    print("A {} is almost the same as a {}.".format(animal, model.most_similar(animal)[0][0]))

A cat is almost the same as a dog.
A dog is almost the same as a cat.
A horse is almost the same as a horses.
A goldfish is almost the same as a crackers.
A lion is almost the same as a dragon.


In [17]:
model.closer_than('man','boy')

['woman']

In [21]:
print(model.distance('man','boy'))
print(model.distance('man','woman'))

0.20851284265518188
0.1676505208015442


And, as we discussed in the lecture, we can literally calculate with the embeddings:

In [27]:
model.most_similar(positive=['woman', 'king'], negative=['man'])

[('queen', 0.7698541283607483),
 ('monarch', 0.6843380928039551),
 ('throne', 0.6755735874176025),
 ('daughter', 0.6594556570053101),
 ('princess', 0.6520534753799438),
 ('prince', 0.6517034769058228),
 ('elizabeth', 0.6464517712593079),
 ('mother', 0.6311717629432678),
 ('emperor', 0.6106470823287964),
 ('wife', 0.6098655462265015)]

In [51]:
# We can do the same by hand, but would need to do some manual cleanup afterwards
# (e.g., removing 'king' itself from the results)
model.similar_by_vector(model.get_vector('king') - model.get_vector('man') + model.get_vector('woman') )

[('king', 0.8551837205886841),
 ('queen', 0.783441424369812),
 ('monarch', 0.6933802366256714),
 ('throne', 0.6833109855651855),
 ('daughter', 0.680908203125),
 ('prince', 0.6713142395019531),
 ('princess', 0.664408266544342),
 ('mother', 0.6579325199127197),
 ('elizabeth', 0.6563301086425781),
 ('father', 0.6392419338226318)]

### TRY YOURSELF

## Using word embeddings in supervised machine learning 

We need to `sudo pip3 install embeddingvectorizer` first.

In [53]:
from glob import glob 
from collections import defaultdict
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
from sklearn.pipeline import Pipeline
import embeddingvectorizer

In [5]:
reviews=[]
test=[]

for file in glob ("/home/damian/Downloads/aclImdb/train/pos/*.txt"):
    with open(file) as fi:
        reviews.append((fi.read(),"1"))
nopostr=len(reviews)
print ("Added",nopostr,"positive reviews")  

for file in glob ("/home/damian/Downloads/aclImdb/train/neg/*.txt"):
    with open(file) as fi:
        reviews.append((fi.read(),"-1"))
nonegtr=len(reviews)-nopostr
print ("Added",nonegtr,"negative reviews")  

for file in glob ("/home/damian/Downloads/aclImdb/test/pos/*.txt"):
    with open(file) as fi:
        test.append((fi.read(),"1"))
noposte=len(test)
print ("Added",noposte,"positive reviews")  

for file in glob ("/home/damian/Downloads/aclImdb/test/neg/*.txt"):
    with open(file) as fi:
        test.append((fi.read(),"-1"))
nonegte=len(test)-noposte
print ("Added",nonegte,"negative reviews")  

Added 12500 positive reviews
Added 12500 negative reviews
Added 12500 positive reviews
Added 12500 negative reviews


First, let's look at our old model

In [13]:
mypipe = Pipeline([('vectorizer', TfidfVectorizer()),
                    ('svm', 
                     SGDClassifier(loss='hinge', penalty='l2', tol=1e-4, alpha=1e-6, max_iter=1000, random_state=42))])

# Generate BOW representation of word counts
mypipe.fit([e[0] for e in reviews], [e[1] for e in reviews])
predictions = mypipe.predict([e[0] for e in test])

print('Precision:')
print(metrics.precision_score([e[1] for e in test],predictions,pos_label='1', labels = ['-1','1']))
print('Recall:')
print(metrics.recall_score([e[1] for e in test],predictions,pos_label='1', labels = ['-1','1']))

Precision:
0.8598565139409751
Recall:
0.84376


Then, let's compare with the new one.
First, we need to convert our word embedding model to the so-called word2vec format

In [54]:
#bigmodel = api.load("word2vec-google-news-300")
# w2vmodel = dict(zip(bigmodel.index2word, bigmodel.syn0))

w2vmodel = dict(zip(model.index2word, model.syn0))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  after removing the cwd from sys.path.


In [55]:
mypipe = Pipeline([('vectorizer', embeddingvectorizer.EmbeddingTfidfVectorizer(w2vmodel, operator='mean')),
                    ('svm', 
                     SGDClassifier(loss='hinge', penalty='l2', tol=1e-4, alpha=1e-6, max_iter=1000, random_state=42))])

# Generate BOW representation of word counts
mypipe.fit([e[0] for e in reviews], [e[1] for e in reviews])
predictions = mypipe.predict([e[0] for e in test])

print('Precision:')
print(metrics.precision_score([e[1] for e in test],predictions,pos_label='1', labels = ['-1','1']))
print('Recall:')
print(metrics.recall_score([e[1] for e in test],predictions,pos_label='1', labels = ['-1','1']))

Precision:
0.8468904556455856
Recall:
0.64384


Well, these embeddings seem to be crap. Let's use the new ones.

In [58]:
bigmodel = api.load("word2vec-google-news-300")
w2vmodel = dict(zip(bigmodel.index2word, bigmodel.syn0))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  


In [59]:
mypipe = Pipeline([('vectorizer', embeddingvectorizer.EmbeddingTfidfVectorizer(w2vmodel, operator='mean')),
                    ('svm', 
                     SGDClassifier(loss='hinge', penalty='l2', tol=1e-4, alpha=1e-6, max_iter=1000, random_state=42))])

# Generate BOW representation of word counts
mypipe.fit([e[0] for e in reviews], [e[1] for e in reviews])
predictions = mypipe.predict([e[0] for e in test])

print('Precision:')
print(metrics.precision_score([e[1] for e in test],predictions,pos_label='1', labels = ['-1','1']))
print('Recall:')
print(metrics.recall_score([e[1] for e in test],predictions,pos_label='1', labels = ['-1','1']))

Precision:
0.9180983252296057
Recall:
0.67976


## Soft cosine similarity

Finally, let's explore soft cosine similarity by re-using our model `model` from above, and our movie reviews `reviews`.

In [69]:
from gensim.similarities import SoftCosineSimilarity, SparseTermSimilarityMatrix
from gensim.corpora import Dictionary
from gensim.models import WordEmbeddingSimilarityIndex

termsim_index = WordEmbeddingSimilarityIndex(model.wv)
documents = [e[0].lower().split() for e in reviews[:100]]

id2word = Dictionary(documents)
bow_corpus = [id2word.doc2bow(document) for document in documents]
similarity_matrix = SparseTermSimilarityMatrix(termsim_index, id2word)  # construct similarity matrix
docsim_index = SoftCosineSimilarity(bow_corpus, similarity_matrix, num_best=10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """


In [71]:
query = '''Pulp Fiction may be the single best film ever made, and quite appropriately it is by one of the most 
creative directors of all time, Quentin Tarantino. This movie is amazing from the beginning definition of pulp to
the end credits and boasts one of the best casts ever assembled with the likes of Bruce Willis, Samuel L. Jackson, 
John Travolta, Uma Thurman, Harvey Keitel, Tim Roth and Christopher Walken. The dialog is surprisingly humorous for
this type of film, and I think that's what has made it so successful. Wrongfully denied the many Oscars it was 
nominated for, Pulp Fiction is by far the best film of the 90s and no Tarantino film has surpassed the quality of
this movie (although Kill Bill came close). As far as I'm concerned this is the top film of all-time and definitely 
deserves a watch if you haven't seen it.
'''.lower().split()
sims = docsim_index[dictionary.doc2bow(query)]  

In [72]:
sims

[(20, 0.9686975479125977),
 (50, 0.9650362133979797),
 (41, 0.9644956588745117),
 (1, 0.9642953872680664),
 (31, 0.9641914367675781),
 (13, 0.9631236791610718),
 (37, 0.9629360437393188),
 (17, 0.9595888257026672),
 (85, 0.9594312310218811),
 (68, 0.959321916103363)]

In [75]:
" ".join(documents[50])

"I was blown away by the re-imagined Battlestar Galactica, a show that always kept me guessing and brought me to tears on more than one occasion. A hardened sci-fi fan, I like to think I can pick out the good stuff from the BS, and this was good stuff.<br /><br />As such, when I first heard about the prospect of a prequel series some months ago I got a sick feeling in my gut. I was afraid that the formula that made Battlestar so successful would be reused in Caprica, which wouldn't work at all. BSG's story, of a mournful ragged band of survivors, trapped aboard decaying star ships and guided by prophetic vision and a sequence of pseudo-miracles, was perfectly complimented by extraordinary music and a better cast of actors.<br /><br />Caprica feels different. Where BSG takes place after the fall of a great civilization, Caprica portrays that civilization in it's cold and decadent heyday. The overall vibe I got from Caprica was similar to that of Minority Report, minus excessive and coun